In [15]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

from dash import Dash, html, dcc, dash_table
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

In [16]:
kreise_df.head()

,Bundesland,NAME_2,Landkreis,Land_Stadt,Zahl_Ohren,Zahl_PV,avg_Score
0,Niedersachsen,Weser-Ems,Oldenburg,Rural district,29,327,0.41
1,Niedersachsen,Weser-Ems,Osnabrück Städte,Urban district,24,355,4.12
2,Niedersachsen,Weser-Ems,Osnabrück,Rural district,45,616,1.77
3,Niedersachsen,Weser-Ems,Vechta,Rural district,28,290,5.53
4,Niedersachsen,Weser-Ems,Wesermarsch,Rural district,42,518,2.25


In [17]:
# import data
kreise_df = pd.read_csv('/home/jan/Uni/DS-Project/modules/dashboard/test/assets/kreise_df.csv')
# rename and drop columns
kreise_df = kreise_df.drop(columns=['Unnamed: 0']).rename(columns={'NAME_1': 'Bundesland', 'NAME_3': 'Landkreis', 'ENGTYPE_3': 'Land_Stadt'})

kreise_df.head()

# good if there are many options
Bundesland_unique = kreise_df['Bundesland'].unique()
Bundesland_options = [{'label': item, 'value': item} for item in np.sort(kreise_df['Bundesland'].unique())]

In [18]:
# heatmap

def discrete_background_color_bins(df, n_bins=5, columns='all'):
    import colorlover
    bounds = [i * (1.0 / n_bins) for i in range(n_bins + 1)]
    if columns == 'all':
        if 'id' in df:
            df_numeric_columns = df.select_dtypes('number').drop(['id'], axis=1)
        else:
            df_numeric_columns = df.select_dtypes('number')
    else:
        df_numeric_columns = df[columns]
    df_max = df_numeric_columns.max().max()
    df_min = df_numeric_columns.min().min()
    ranges = [
        ((df_max - df_min) * i) + df_min
        for i in bounds
    ]
    styles = []
    legend = []
    for i in range(1, len(bounds)):
        min_bound = ranges[i - 1]
        max_bound = ranges[i]
        backgroundColor = colorlover.scales[str(n_bins)]['seq']['Greens'][i - 1]
        color = 'white' if i > len(bounds) / 2. else 'inherit'

        for column in df_numeric_columns:
            styles.append({
                'if': {
                    'filter_query': (
                        '{{{column}}} >= {min_bound}' +
                        (' && {{{column}}} < {max_bound}' if (i < len(bounds) - 1) else '')
                    ).format(column=column, min_bound=min_bound, max_bound=max_bound),
                    'column_id': column
                },
                'backgroundColor': backgroundColor,
                'color': color
            })
        legend.append(
            html.Div(style={'display': 'inline-block', 'width': '60px'}, children=[
                html.Div(
                    style={
                        'backgroundColor': backgroundColor,
                        'borderLeft': '1px rgb(50, 50, 50) solid',
                        'height': '10px'
                    }
                ),
                html.Small(round(min_bound, 2), style={'paddingLeft': '2px'})
            ])
        )

    return (styles, html.Div(legend, style={'padding': '5px 0 5px 0'}))

# run function
(styles, legend) = discrete_background_color_bins(kreise_df, columns=['avg_Score'])

In [19]:
app = Dash(__name__, external_stylesheets = [dbc.themes.LITERA])

# change to app.layout if running as single page app instead
app.layout = html.Div([
    dbc.Container([
        dbc.Row([
            dbc.Col(html.H1(children='Alle Landkreise auf einen Blick'), className="mb-2")
        ]),
        dbc.Row([
            dbc.Col(html.H6(children='die wichtigsten Kennzahlen zur Filterung und zum Nachschlagen.'), className="mb-4")
        ]),
# choose between Bundesland
    dcc.Dropdown(
        id='Bundesland_choice',
        options=Bundesland_options,
        value=None,
        #multi=True,
        style={'width': '50%'}
        ),
# define table
    dbc.Row([
        dash_table.DataTable(
            id='table',
            # data
            columns = [{"name": i, "id": i} for i in kreise_df.columns],
            data=kreise_df.to_dict('records'),
            # sorting
            sort_action = 'native',
            sort_mode = 'multi',
            # pagination
            page_current=0,
            page_size=20,
            page_action='native',
            # style
            style_data_conditional = styles,
            style_cell_conditional=[{
                'if': {'column_id': c},
                'textAlign': 'left'
                } for c in ['Bundesland', 'Landkreis', 'Land_Stadt']],
            style_as_list_view=True,
        ),
        legend])
])])

@app.callback(
    Output('table', 'data'),
    [Input('Bundesland_choice', 'value')]
)
def update_table(Bundesland_choice):
    if Bundesland_choice:
        filtered_df = kreise_df[kreise_df['Bundesland'] == Bundesland_choice]
    else:
        filtered_df = kreise_df
    return filtered_df.to_dict(orient='records')

In [20]:
# run app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [13/Feb/2023 16:35:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 16:35:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 16:35:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 16:35:02] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 16:35:02] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 16:35:02] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 16:35:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 16:35:43] "POST /_dash-update-component HTTP/1.1" 200 -


# Test Bereich